In [ ]:
# libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# making dataframe from UN Energy data
all_countries_df = pd.read_csv("resources/UNdata_Energy_PowerPlant.csv")

# making dataframe from scrapped african nations
african_countries_df = pd.read_csv("resources/african_nations.csv")

In [ ]:
print("All nations: ", all_countries_df.info())
print("African nations: ", african_countries_df.info())

### Cleaning Operations

In [ ]:
## Dropping irelevant rows and columns
all_countries_df.drop([65988, 65989], inplace=True) 
all_countries_df.drop("Quantity Footnotes", axis=1, inplace=True)

In [ ]:
# renaming column names for easier referencing
all_countries_df.rename(columns={"Country or Area":"country_or_area", "Commodity - Transaction":"commodity_transaction", "Year":"year", "Unit":"unit", "Quantity":"quantity"}, inplace=True)

In [ ]:
all_countries_df.head()

In [ ]:
## preparing dataset for merge with all nations

# renaming country column on african_countries_df
african_countries_df.rename(columns={"Country":"country_or_area"}, inplace=True)

In [ ]:
# renaming country names in general dataframe to match name in african nations df
all_countries_df.country_or_area.replace({"United Rep. of Tanzania":"Tanzania", "Dem. Rep. of the Congo":"DR Congo", 
                                    "Sao Tome and Principe":"Sao Tome & Principe", "Central African Rep.":"Central African Republic"}, inplace=True)

#### Merging datasets

In [ ]:
africa_plants = all_countries_df.merge(african_countries_df, on="country_or_area")

#### Verifying state of new dataset with further cleaning

In [ ]:
print(africa_plants.info())

In [ ]:
africa_plants.duplicated().sum()

In [ ]:
africa_plants.describe()

In [ ]:
# reformatting year column to workable string values

africa_plants.year = africa_plants.year.astype(str)
africa_plants.year = africa_plants.year.apply(lambda x: x.split('.')[0])

<small>year column was converted to string instead of datetime column because incorrect years were obtained using datetime method</small>

In [ ]:
# fixing solar entires
africa_plants["commodity_transaction"].replace({"Electricity - net installed capacity of electric power plants public solar":
                                        "Electricity - net installed capacity of electric power plants, public solar"}, inplace=True)

In [ ]:
# subsetting dataset of reported net capacity of plants
africa_net_capacity_plants = africa_plants.copy()
africa_net_capacity_plants = africa_net_capacity_plants[africa_net_capacity_plants.commodity_transaction.str.contains(",")]
# only net capacity electric power plants contains commas in dataset

In [ ]:
# getting fuel source used in power plants
africa_net_capacity_plants.commodity_transaction = africa_net_capacity_plants.commodity_transaction.apply(lambda x:x.split(',')[1])

In [ ]:
# unique energy sources
africa_net_capacity_plants.commodity_transaction.unique()

In [ ]:
# removing space before actual value
africa_net_capacity_plants.commodity_transaction = africa_net_capacity_plants.commodity_transaction.apply(lambda x:x[1:])

In [ ]:
# merging same fuel sources even if they are of public or private generation, exempting self-producing plants, however.

africa_net_capacity_plants.commodity_transaction.replace({"public combustible fuels":"combustible fuels", 
                                        "public hydro":"hydro", "public wind":"wind", "public nuclear":"nuclear", 
                                        "public geothermal":"geothermal", "public solar":"solar"}, inplace=True)

<small>Before merging of fuels I discovered that same fuel sources had duplicated records. Merging the fuels revealed this abnormally</small>

In [ ]:
africa_net_capacity_plants.duplicated().sum()

In [ ]:
# dropping duplicate records
africa_net_capacity_plants.drop_duplicates(inplace=True)
africa_net_capacity_plants.nunique()

### Univariate Plots

distribution of quantity

In [ ]:
plt.figure(figsize=[14.70,8.27])

plt.hist(data=africa_net_capacity_plants, x='quantity', bins=100)
plt.xlabel('Quantity (kW, thousand)', fontdict={'fontsize':13})
plt.yscale('log')
plt.yticks([1e0, 3e0, 1e1, 3e1, 1e2, 3e2, 1e3, 3e3, 1e4], [1, 3, 10, 30, 100, 300, '1k', '3k', '10k'])

plt.title("Frequency Distribution of Quantity", fontdict={'fontsize':15})
plt.show()

* The Quantity variable is of unimodal distribution
* Most of the power plants have generation capacities between 0 (kW, thousand) to 10,000 (kW, thousand). 

In [ ]:
plt.figure(figsize=[14.70,8.27])

plt.hist(data=africa_net_capacity_plants, x='quantity', bins=100)
plt.xlabel('Quantity (kW, thousand)', fontdict={'fontsize':13})
plt.yscale('log')
plt.yticks([1e0, 3e0, 1e1, 3e1, 1e2, 3e2, 1e3, 3e3, 1e4], [1, 3, 10, 30, 100, 300, '1k', '3k', '10k'])
plt.xscale('log')
plt.xticks([1e0, 2e0, 5e0, 1e1, 2e1, 5e1, 1e2, 2e2, 5e2, 1e3, 2e3, 5e3, 1e4, 2e4, 5e4, 1e5], 
            [1, 2, 5, 10, 20, 50, 100, 200, 500, '1k', '2k', '5k', '10k', '20k', '50k', '100k'])

plt.title("Frequency Distribution of Quantity", fontdict={'fontsize':15})
plt.show()

* Closer looks reveals plants delivering capacities of 0 (kW, thousand) to >500 (kW, thousand) are of the modal class, there is a steep down and steady decline afterwards

In [ ]:
# general_plants df stripped of self-producer, autoproducer & main activity commodities
general_plants = africa_net_capacity_plants[~(africa_net_capacity_plants.commodity_transaction.str.contains("self-producer|autoproducer|main"))]

In [ ]:
general_plants.commodity_transaction.value_counts()

In [ ]:
# countplot of energy sources
plt.figure(figsize=[14.70, 8.27])
# sns.countplot(x='commodity_transaction', data=general_plants, color=sns.color_palette()[0])
general_plants.commodity_transaction.value_counts(ascending=False).plot.bar()
plt.xticks(rotation=0)
plt.show()

* Combustible fuels power plants are the most numerous on the continent, followed by Hydro which makes sense seeing those are the technologies, she started with.

### Bivariate Plots

In [ ]:
# Plants vs. Quantity
plt.figure(figsize=[14.70, 8.27])

sns.barplot(x='quantity', y='commodity_transaction', data=africa_net_capacity_plants, 
            color=sns.color_palette()[0], order=africa_net_capacity_plants.commodity_transaction.value_counts(ascending=False).index)
plt.show()

In [ ]:
# Quantity vs. Fuels
plt.figure(figsize=[14.70, 8.27])
sns.barplot(data=general_plants, x='commodity_transaction', y='quantity', color=sns.color_palette()[0], order=general_plants.commodity_transaction.value_counts(ascending=False).index)

plt.show()

<small>
Observation:
<ul>
<li>Main activity plants and a combination of main activity and autoproducer plants make up highest electricity productions,
but sadly, energy source used isn't specified</li>
<li>Nuclear and combustibles make the highest productions for known sources</li>
</ul>
</small>

### Multivariate Explorations

#### Functions

In [ ]:
general_plants_years = general_plants.copy()

def sum_quantity(year, type, old_df):
    """
    creates a new_df from specified year of old_df
    sums the quantity for every commodity_transaction for that particular year
    """
    new_df = old_df.query("year == @year")
    if type == "commodity":
        new_df = new_df.groupby(['year', 'commodity_transaction']).sum()['quantity'].reset_index()
    elif type == "country":
        new_df = new_df.groupby(['year', 'country_or_area']).sum()['quantity'].reset_index()
        
    return new_df

In [ ]:
def create_year_dfs(df, type):
    """
    creates a list of plants years 1990-2020 with an interval year of 3 all from df
    df: dataframe containing data on commodity_transaction from 1990-2020
    """
    plants_list = []
    year = 1990
    for i in range(11):
        new_plant = sum_quantity(type=type, year=str(year), old_df = df)
        year += 3
        plants_list.append(new_plant)
    return plants_list

In [ ]:
plant_year_list = create_year_dfs(df=general_plants, type="commodity")

plants_year_dist = pd.concat(plant_year_list, ignore_index=True)

In [ ]:
plants_year_dist.sample(7)

#### Plots

In [ ]:
# quantity trend of energy plants

plt.figure(figsize=[14.70, 8.27])
plt.style.use('seaborn-whitegrid')
sns.lineplot(
    x='year',
    y='quantity',
    hue='commodity_transaction',
    palette= 'bright',
    style='commodity_transaction',
    markers=True,
    dashes=False,
    data=plants_year_dist
)

plt.yscale('log')
plt.yticks([3e0, 1e1, 3e1, 1e2, 3e2, 1e3, 3e3, 1e4, 3e4, 1e5, 3e5], 
            [3, 10, 30, 100, 300, 1000, 3000, '10K', '30K', '100K', '300K']);
plt.ylim([0.1e1, 5e5])
plt.legend(title='Fuels')
plt.show()

In [ ]:
self_producer_plants  = africa_net_capacity_plants[africa_net_capacity_plants.commodity_transaction.str.contains("self")]
self_producer_plants.commodity_transaction.value_counts()

In [ ]:
sp_plant_year_list = create_year_dfs(df=self_producer_plants, type="commodity")
sp_plants_year_dist = pd.concat(sp_plant_year_list, ignore_index=True)

In [ ]:
# quantity trend of self-producer plants
plt.figure(figsize=[14.70, 8.27])
plt.style.use('seaborn-whitegrid')
sns.lineplot(
    x='year',
    y='quantity',
    hue='commodity_transaction',
    palette= 'bright',
    style='commodity_transaction',
    markers=True,
    dashes=False,
    data=sp_plants_year_dist
)

plt.yscale('log')
plt.yticks([1e0, 2e0, 5e0, 1e1, 2e1, 5e1, 1e2, 2e2, 5e2, 1e3, 2e3, 5e3, 1e4, 2e4], 
            [1, 2, 5, 10, 20, 50, 100, 200, 500, '1k', '2k', '5k', '10k', '20k']);
# plt.ylim([0.1e1, 5e5])
plt.legend(['combustible fuels', 'hydro', 'geothermal', 'solar', 'wind'], title='Fuels', loc=4, fontsize=12)
plt.show()

### Exploration of Africa's Top 5 (GDP)

In [ ]:
# subsetting top 5 nations
top5_gdp = general_plants[general_plants.country_or_area.str.contains("Nigeria|Egypt|South Africa|Algeria|Morocco")]

In [ ]:
# getting records from 1990 - 2020 with interval of 3 years
top5_gdp_plant_list = create_year_dfs(df=top5_gdp, type="country")

In [ ]:
# joining year records
top5_gdp_year_dist = pd.concat(top5_gdp_plant_list, ignore_index=True)
top5_gdp_year_dist.head()

#### Plots

In [ ]:
# quantity trend of energy plants in africa's top 5
plt.figure(figsize=[14.70, 8.27])
plt.style.use('seaborn-whitegrid')
sns.lineplot(
    x='year',
    y='quantity',
    hue='country_or_area',
    palette= 'bright',
    style='country_or_area',
    markers=True,
    dashes=False,
    data=top5_gdp_year_dist
)

plt.yscale('log')
plt.yticks([1e3, 2e3, 5e3, 1e4, 2e4, 5e4, 1e5, 2e5], 
            ['1k', '2k', '5k', '10k', '20k', '50k', '100k', '200k']);
plt.ylim([2.5e3, 2e5])
plt.legend(title='Country')
plt.show()

In [ ]:
# records of top 5 from 1990-2019
top5_gdp_total_quantities = top5_gdp.query("year >='1990' and year <='2019'")
# excluded year 2020 because only Egypt has records for that year

# dataset of total quantity of all energy plants in the top 5
top5_gdp_total_quantities = top5_gdp_total_quantities.groupby(['country_or_area']).sum()['quantity'].reset_index()

In [ ]:
# total quantity vs. country
plt.figure(figsize=[14.70, 8.27])

plt.bar(x='country_or_area', height='quantity', data=top5_gdp_total_quantities)
plt.xlabel("Country", fontsize=13)
plt.xticks(fontsize=11)
plt.ylabel("Quantity (kW, thousand)", fontsize=13)
plt.yticks([5e5, 1e6, 1.5e6, 2e6, 2.5e6], ['500k', '1m', '1.5m', '2m', '2.5m'], fontsize=11)
plt.title("Total Net Generated Electricity from Plants in Africa's Top 5 (GDP)", fontsize=15)
plt.show()

In [ ]:
# records of top 5 from 1990-2019
top5_gdp_mean_quantity = top5_gdp.query("year >='1990' and year <='2019'")
# excluded year 2020 because only Egypt has records for that year

# dataset of total number of plants and average quantity in the top 5
top5_gdp_mean_quantity = top5_gdp_mean_quantity.groupby('country_or_area') \
       .agg(count=('country_or_area', 'size'), avg_quantity=('quantity', 'mean')) \
       .reset_index()

In [ ]:
# avg_quantity/no._of_plants vs. country
ax = top5_gdp_mean_quantity.plot(x='country_or_area', y=['avg_quantity', 'count'], kind="bar", figsize=[14.70, 8.27], colormap='coolwarm')
for container in ax.containers:
    ax.bar_label(container, padding=5, fontsize=12)

plt.xticks(rotation=0, fontsize=12)
plt.xlabel("Country", fontsize=13)

plt.yscale('log')
plt.yticks([1e2, 2e2, 5e2, 1e3, 2e3, 5e3, 1e4, 2e4], [100, 200, 500, '1k', '2k', '5k', '10k', '20k'], fontsize=11)

plt.title("Average Net Electricty Generated and Number of Plants in Africa's Top 5 (GDP)", fontsize=15)
plt.legend(["Avg. Quantity (kw, thousand)", "No. of Plants"], fontsize=12)

plt.show()

Key Observations:
* Algeria has the least amount of plants but better average generating capacity than Morocco and Nigeria
* Nigeria and Morocco seems to have plants that are either smaller or less efficient compared with counterpart nations
* South Africa seems to posses bigger or more efficient plants in their numbers, likewise Egypt, but with fewer plants

In [ ]:
# pivot table of number of unique plants in the top 5
top5_technologies = top5_gdp.groupby(['country_or_area', 'commodity_transaction']).size().reset_index().pivot(columns='commodity_transaction', index='country_or_area', values=0)

In [ ]:
# num_of_plants/energy_sources vs. country
my_cmap = sns.color_palette('deep', as_cmap=True)
top5_technologies.plot(kind='bar', stacked=True, color=my_cmap, figsize=[14.70, 8.27]);
plt.xlabel('Country', fontsize=13)
plt.ylabel('Number of plants', fontsize=13);
plt.xticks(rotation=0, fontsize=12);
plt.yticks(fontsize=12)
plt.title('Number of Energy Plants Employed by Africa\'s Top 5 (GDP)', fontsize=15);
plt.legend(title='Energy Sources', frameon=False, fontsize=12);
sns.despine(bottom=True, left=True)

Key Observations:
* Combustible fuels have an all time high across all countries
* Egypt and Morocco engage more in wind energy generation than counterpart nations
* Nigeria has more hydro plants than counterpart nations with a number higher than her oil & gas plants
* South Africa has done more work than in having a diversified energy mix than counterpart nations. Both its hydro and combustible fuels plants are roughly equal in number.
* South Africa is the only nation with a Nuclear plant, with a number (30) rivaling energy plants of counterpart nations

In [ ]:
# total capacity for technologies used in the top 5
top5_powerplants_quantity = top5_gdp.groupby(['country_or_area', 'commodity_transaction']).sum()['quantity'].reset_index()


In [ ]:
# capacities of unique plants in the top 5
plt.figure(figsize=[14.70, 8.27])
sns.barplot(data=top5_powerplants_quantity, x='country_or_area', y='quantity', hue='commodity_transaction');

plt.yscale('log');
plt.xlabel('Country', fontsize=13);
plt.xticks(fontsize=12)

plt.ylabel('Quantity (kW, thousand)', fontsize=13);
plt.yticks([3e1, 1e2, 3e2, 1e3, 3e3, 1e4, 3e4, 1e5, 3e5, 1e6], 
            [30, 100, 300, 1000, 3000, '10K', '30K', '100K', '300K', '1M'], fontsize=12);

plt.title('Total Net Electricty Generated by Africa\'s Top 5 (GDP) Energy Sources', fontsize=15);
# plt.legend(frameon=False, fontsize=12);
plt.legend(bbox_to_anchor =(0.86, 0.75), title = "Energy Sources", frameon=False, fontsize=12);
sns.despine(left=True, bottom=True)

Key Observations: 
* Among the top 5, South Africa is the only country employing nuclear energy, which has productions higher than average as compared with other prominent technologies
* South Africa seems to have an diversified energy mix almost in equal proportion save combustibles
* Hydro technology apart from combustibles, seems to be the other source of generation the power economy the top 5's solidly relies on
* Algeria and Nigeria seem to be the only one's in the top 5 using so little of wind energy